# Statistical Models

- Run Mixed Effect Linear Models

In [17]:
import json
#Pandas for saving datasets
import pandas as pd
#matplotlib for rendering
import matplotlib.pyplot as plt
#numpy for handeling matrix operations
import numpy as np
#time, to, well... keep track of time
import time
#iPython display for making sure we can render the frames
from IPython import display
#seaborn for rendering
import seaborn
import math
import os
import statistics as stat
import glob
import seaborn as sns
import pandas as pd
import itertools


In [4]:
#code from scipy documentation
from scipy.stats import t
from scipy.stats import norm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf
import pingouin as pg


# Load Data

In [5]:
# Get data file names
cur_path = os.getcwd()
filenames = glob.glob(cur_path + "/data/*.csv")
# Read dframes into a list
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# Filter data

In [6]:
# get first dataframe and filter columns

filt_dfs = []
# concat all dframes into one dframe
for sub_num, d in enumerate(dfs):
    my_df = d.filter(items=['congruent', 'Rand Tim', 'key_resp.rt','block_num'])
    my_df['subject_number'] = [sub_num] * len(my_df)  # Add subject number to the DataFrame

    # drop na, only 10s
    filt_dfs.append(my_df.dropna())

## concat #
full_dat = pd.concat(filt_dfs)


In [7]:
print(f'num subs: {len(dfs)}')

num subs: 26


In [10]:
full_dat

,congruent,Rand Tim,key_resp.rt,block_num,subject_number
1,0.0,1.3132,0.636110,5s,0
2,1.0,0.7037,0.709031,5s,0
3,0.0,0.8309,0.434464,5s,0
4,1.0,4.6764,0.553009,5s,0
5,0.0,4.6441,0.518856,5s,0
...,...,...,...,...,...
416,1.0,4.1302,0.473883,5s,25
419,0.0,4.1825,0.864461,5s,25
420,0.0,3.5785,0.386026,5s,25
421,0.0,1.9596,0.395578,5s,25


# Fig 3: Spatial and Temporal Effects

In [11]:
# Create a DataFrame
df = pd.DataFrame(full_dat)

# Rename columns for clarity
df.rename(columns={
    'congruent': 'condition_type',
    'Rand Tim': 'stimulus_onset',
    'key_resp.rt': 'reaction_time',
    'block_num': 'task'
}, inplace=True)

# Convert categorical variables to the appropriate type
df['condition_type'] = df['condition_type'].astype('category')
#df['task'] = df['task'].astype('category')
df['task'] = pd.Categorical(df['task'], categories=['1s', '3s', '5s', '10s'], ordered=True)  # Set reference condition here


# Build the fixed effects model
model = smf.mixedlm('reaction_time ~ C(task) * C(condition_type) * stimulus_onset', 
                    data=df, 
                    groups=df['subject_number']).fit()
# Display the model summary
print(model.summary())

                              Mixed Linear Model Regression Results
Model:                          MixedLM             Dependent Variable:             reaction_time
No. Observations:               9350                Method:                         REML         
No. Groups:                     26                  Scale:                          0.0098       
Min. group size:                250                 Log-Likelihood:                 8223.9148    
Max. group size:                394                 Converged:                      Yes          
Mean group size:                359.6                                                            
-------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------
Intercept                                         

/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


## Posthoc Tests

In [ ]:
'''
Regression models do not automatically compute all pairwise differences between levels of a factor. 
Instead, they compare each level to a **base (reference) level**, which is chosen by default or set manually.

Our goal is to test whether the effect of condition_type (valid vs. invalid) on reaction time varies 
across levels of task (1s, 3s, 5s, 10s) — i.e., to assess a **two-way interaction**.

There are two main ways to test whether the size of the cueing effect (valid vs. invalid difference) 
changes across tasks:

1. **Marginal Means / Difference Scores Approach**:
   - For each subject and task level, compute the difference in reaction time between invalid and 
   valid trials.
   - This gives you a table of cueing effects (one per subject per task).
   - Then, perform a repeated-measures ANOVA on these difference scores to test 
   whether the cueing effect differs across tasks.
   - This tests the **main effect of task on cueing differences**, but doesn't 
   give pairwise comparisons unless you run post hoc tests.

2. **Regression Modeling Approach**:
   - Fit a mixed-effects model including the interaction: `reaction_time ~ C(task) * C(condition_type)`.
   - The model estimates how reaction time is influenced by task, condition_type, and their interaction, 
   controlling for repeated measures via subject-level random effects.
   - Coefficients like `C(task)[T.5s]:C(condition_type)[T.invalid]` 
   tell you how the cueing effect differs in the 5s task compared to the reference task (e.g., 1s).
   - For example:
     - A coefficient of `-0.019` means the invalid-valid difference is ~0.019 **smaller** in the 10s 
     task than in the 1s task.
   - To test other pairwise task comparisons (e.g., 3s vs 5s), you can relevel the `task` variable and refit the model.

Importantly, regression gives standard errors and p-values, allowing for hypothesis testing. 
Even if you can manually compute cueing differences, the model tells you whether those differences 
are statistically meaningful.

Both approaches are valid. The marginal means route gives a
per-subject cueing effect that aligns well with experimental reporting. 
The regression model provides a full inferential framework with flexible control of covariates and variance.

In summary:
- Use **marginal means + ANOVA** to assess whether cueing effects vary across tasks.
- Use **regression models** to formally test interactions and estimate how much larger/smaller the
cueing effect is at each task level, compared to a reference.
- We decided to use regression models to test RT differences between each task. For each posthoc analysis, we reset the
  base condition to identify RT between all levels of the task. The default - 1s - compares 1s - 3s, 1s - 5s, 1s -10s,
  but not 3s - 5s, etc..
'''

### 3s base

In [26]:
df['task'] = pd.Categorical(df['task'], categories=[ '3s', '5s', '10s','1s'], ordered=True)  # Set reference condition here


# Build the fixed effects model
model = smf.mixedlm('reaction_time ~ C(task) * C(condition_type) * stimulus_onset', 
                    data=df, 
                    groups=df['subject_number']).fit()
print(model.summary())

                              Mixed Linear Model Regression Results
Model:                          MixedLM             Dependent Variable:             reaction_time
No. Observations:               9350                Method:                         REML         
No. Groups:                     26                  Scale:                          0.0098       
Min. group size:                250                 Log-Likelihood:                 8223.9148    
Max. group size:                394                 Converged:                      Yes          
Mean group size:                359.6                                                            
-------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------
Intercept                                         

/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


### 5s base

In [30]:
df['task'] = pd.Categorical(df['task'], categories=[ '5s', '3s', '10s','1s'], ordered=True)  # Set reference condition here


# Build the fixed effects model
model = smf.mixedlm('reaction_time ~ C(task) * C(condition_type) * stimulus_onset', 
                    data=df, 
                    groups=df['subject_number']).fit()
print(model.summary())

                              Mixed Linear Model Regression Results
Model:                          MixedLM             Dependent Variable:             reaction_time
No. Observations:               9350                Method:                         REML         
No. Groups:                     26                  Scale:                          0.0098       
Min. group size:                250                 Log-Likelihood:                 8223.9148    
Max. group size:                394                 Converged:                      Yes          
Mean group size:                359.6                                                            
-------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------
Intercept                                         

/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


# Fig 4: Temporal effects across tasks (conditioned on in/valid trials)

### 1 s Base

In [97]:
# Create a DataFrame
df = pd.DataFrame(full_dat)

# Rename columns for clarity
df.rename(columns={
    'congruent': 'condition_type',
    'Rand Tim': 'stimulus_onset',
    'key_resp.rt': 'reaction_time',
    'block_num': 'task'
}, inplace=True)

### Valid or Invalid Trials
df = df[df['condition_type'] == 0] ### 0 = valid, 1 = invalid ** CHANGE ME

# Convert categorical variables to the appropriate type
df['condition_type'] = df['condition_type'].astype('category')
#df['task'] = df['task'].astype('category')
df['task'] = pd.Categorical(df['task'], categories=['1s', '3s', '5s', '10s'], ordered=True)  # Set reference condition here



# Build the fixed effects model
model = smf.mixedlm('reaction_time ~ C(task)  * stimulus_onset', 
                    data=df, 
                    groups=df['subject_number']).fit()
# Display the model summary
print(model.summary())

                 Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     reaction_time
No. Observations:      7485        Method:                 REML         
No. Groups:            26          Scale:                  0.0090       
Min. group size:       197         Log-Likelihood:         6928.4343    
Max. group size:       318         Converged:              Yes          
Mean group size:       287.9                                            
------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------
Intercept                      0.401    0.009 42.708 0.000  0.383  0.420
C(task)[T.3s]                  0.026    0.007  3.654 0.000  0.012  0.040
C(task)[T.5s]                  0.045    0.007  6.321 0.000  0.031  0.059
C(task)[T.10s]                 0.060    0.007  8.442 0.000  0.046  0.

/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


### 3 s base

In [98]:
# Create a DataFrame
df = pd.DataFrame(full_dat)

# Rename columns for clarity
df.rename(columns={
    'congruent': 'condition_type',
    'Rand Tim': 'stimulus_onset',
    'key_resp.rt': 'reaction_time',
    'block_num': 'task'
}, inplace=True)

### Valid or Invalid Trials
df = df[df['condition_type'] == 0] ### 0 = valid, 1 = invalid ** CHANGE ME

# Convert categorical variables to the appropriate type
df['condition_type'] = df['condition_type'].astype('category')
#df['task'] = df['task'].astype('category')
df['task'] = pd.Categorical(df['task'], categories=['3s', '1s', '5s', '10s'], ordered=True)  # Set reference condition here


# Build the fixed effects model
model = smf.mixedlm('reaction_time ~ C(task) * stimulus_onset', 
                    data=df, 
                    groups=df['subject_number']).fit()
# Display the model summary
print(model.summary())

                 Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     reaction_time
No. Observations:      7485        Method:                 REML         
No. Groups:            26          Scale:                  0.0090       
Min. group size:       197         Log-Likelihood:         6928.4343    
Max. group size:       318         Converged:              Yes          
Mean group size:       287.9                                            
------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------
Intercept                      0.428    0.009 47.012 0.000  0.410  0.445
C(task)[T.1s]                 -0.026    0.007 -3.654 0.000 -0.040 -0.012
C(task)[T.5s]                  0.019    0.007  2.784 0.005  0.005  0.032
C(task)[T.10s]                 0.034    0.007  5.035 0.000  0.020  0.

/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


### 5 s base

In [99]:
# Create a DataFrame
df = pd.DataFrame(full_dat)

# Rename columns for clarity
df.rename(columns={
    'congruent': 'condition_type',
    'Rand Tim': 'stimulus_onset',
    'key_resp.rt': 'reaction_time',
    'block_num': 'task'
}, inplace=True)

### Valid or Invalid Trials
df = df[df['condition_type'] == 0] ### 0 = valid, 1 = invalid ** CHANGE ME

# Convert categorical variables to the appropriate type
df['condition_type'] = df['condition_type'].astype('category')
#df['task'] = df['task'].astype('category')
df['task'] = pd.Categorical(df['task'], categories=['5s', '3s', '1s', '10s'], ordered=True)  # Set reference condition here


# Build the fixed effects model
model = smf.mixedlm('reaction_time ~ C(task) * stimulus_onset', 
                    data=df, 
                    groups=df['subject_number']).fit()
# Display the model summary
print(model.summary())

                  Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     reaction_time
No. Observations:       7485        Method:                 REML         
No. Groups:             26          Scale:                  0.0090       
Min. group size:        197         Log-Likelihood:         6928.4343    
Max. group size:        318         Converged:              Yes          
Mean group size:        287.9                                            
-------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                      0.446    0.009  49.402 0.000  0.428  0.464
C(task)[T.3s]                 -0.019    0.007  -2.784 0.005 -0.032 -0.005
C(task)[T.1s]                 -0.045    0.007  -6.321 0.000 -0.059 -0.031
C(task)[T.10s]                 0.015    0.007   2.282 0.

/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


## Normalized Luminance Onset

### 1 s base

In [102]:
# Create a DataFrame
df = pd.DataFrame(full_dat)

# Rename columns for clarity
df.rename(columns={
    'congruent': 'condition_type',
    'Rand Tim': 'stimulus_onset',
    'key_resp.rt': 'reaction_time',
    'block_num': 'task'
}, inplace=True)

### Valid or Invalid Trials
df = df[df['condition_type'] == 0] ### 0 = valid, 1 = invalid ** CHANGE ME
# Convert categorical variables to the appropriate type
df['condition_type'] = df['condition_type'].astype('category')

#df['task'] = df['task'].astype('category')
df['task'] = pd.Categorical(df['task'], categories=['1s', '3s', '5s', '10s'], ordered=True)  # Set reference condition here
df['task_numerical'] = df['task'].str.replace('s', '', regex=False).astype(float)
df['norm_onset'] = df['stimulus_onset'] / df['task_numerical']

### Valid or Invalid Trials
df[df['condition_type'] == 0] ### 0 = valid, 1 = invalid ** CHANGE ME

# Build the fixed effects model
model = smf.mixedlm('reaction_time ~ C(task) * norm_onset', 
                    data=df, 
                    groups=df['subject_number']).fit()
# Display the model summary
print(model.summary())

               Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    reaction_time
No. Observations:    7485       Method:                REML         
No. Groups:          26         Scale:                 0.0090       
Min. group size:     197        Log-Likelihood:        6933.4449    
Max. group size:     318        Converged:             Yes          
Mean group size:     287.9                                          
--------------------------------------------------------------------
                          Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------
Intercept                  0.401    0.009 42.708 0.000  0.383  0.420
C(task)[T.3s]              0.026    0.007  3.654 0.000  0.012  0.040
C(task)[T.5s]              0.045    0.007  6.321 0.000  0.031  0.059
C(task)[T.10s]             0.060    0.007  8.442 0.000  0.046  0.074
norm_onset                -0.087    0.009 -9.943 0

/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


### 3 s base

In [87]:
# Create a DataFrame
df = pd.DataFrame(full_dat)

# Rename columns for clarity
df.rename(columns={
    'congruent': 'condition_type',
    'Rand Tim': 'stimulus_onset',
    'key_resp.rt': 'reaction_time',
    'block_num': 'task'
}, inplace=True)

### Valid or Invalid Trials
df = df[df['condition_type'] == 0] ### 0 = valid, 1 = invalid ** CHANGE ME

# Convert categorical variables to the appropriate type
df['condition_type'] = df['condition_type'].astype('category')
#df['task'] = df['task'].astype('category')
df['task'] = pd.Categorical(df['task'], categories=['3s', '1s', '5s', '10s'], ordered=True)  # Set reference condition here
df['task_numerical'] = df['task'].str.replace('s', '', regex=False).astype(float)
df['norm_onset'] = df['stimulus_onset'] / df['task_numerical']


# Build the fixed effects model
model = smf.mixedlm('reaction_time ~ C(task)  * norm_onset', 
                    data=df, 
                    groups=df['subject_number']).fit()
# Display the model summary
print(model.summary())

               Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    reaction_time
No. Observations:    7485       Method:                REML         
No. Groups:          26         Scale:                 0.0090       
Min. group size:     197        Log-Likelihood:        6933.4449    
Max. group size:     318        Converged:             Yes          
Mean group size:     287.9                                          
--------------------------------------------------------------------
                          Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------
Intercept                  0.428    0.009 47.012 0.000  0.410  0.445
C(task)[T.1s]             -0.026    0.007 -3.654 0.000 -0.040 -0.012
C(task)[T.5s]              0.019    0.007  2.784 0.005  0.005  0.032
C(task)[T.10s]             0.034    0.007  5.035 0.000  0.020  0.047
norm_onset                -0.075    0.008 -9.350 0

/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


### 5 s base

In [86]:
# Create a DataFrame
df = pd.DataFrame(full_dat)

# Rename columns for clarity
df.rename(columns={
    'congruent': 'condition_type',
    'Rand Tim': 'stimulus_onset',
    'key_resp.rt': 'reaction_time',
    'block_num': 'task'
}, inplace=True)

### Valid or Invalid Trials
df = df[df['condition_type'] == 0] ### 0 = valid, 1 = invalid ** CHANGE ME

# Convert categorical variables to the appropriate type
df['condition_type'] = df['condition_type'].astype('category')
#df['task'] = df['task'].astype('category')
df['task'] = pd.Categorical(df['task'], categories=['5s', '3s', '1s', '10s'], ordered=True)  # Set reference condition here
df['task_numerical'] = df['task'].str.replace('s', '', regex=False).astype(float)
df['norm_onset'] = df['stimulus_onset'] / df['task_numerical']


# Build the fixed effects model
model = smf.mixedlm('reaction_time ~ C(task)  * norm_onset', 
                    data=df, 
                    groups=df['subject_number']).fit()
# Display the model summary
print(model.summary())

                Mixed Linear Model Regression Results
Model:                MixedLM    Dependent Variable:    reaction_time
No. Observations:     7485       Method:                REML         
No. Groups:           26         Scale:                 0.0090       
Min. group size:      197        Log-Likelihood:        6933.4449    
Max. group size:      318        Converged:             Yes          
Mean group size:      287.9                                          
---------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
Intercept                  0.446    0.009  49.402 0.000  0.428  0.464
C(task)[T.3s]             -0.019    0.007  -2.784 0.005 -0.032 -0.005
C(task)[T.1s]             -0.045    0.007  -6.321 0.000 -0.059 -0.031
C(task)[T.10s]             0.015    0.007   2.282 0.022  0.002  0.028
norm_onset                -0.082    

/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


## Fig 5: Nonlinear Temporal Effects

- Construct a mixed effects model for the first or second half of each task. 

In [106]:
# which half of the task? 
half = 'second' # first or second

# Create a DataFrame
df = pd.DataFrame(full_dat)

# Rename columns for clarity
df.rename(columns={
    'congruent': 'condition_type',
    'Rand Tim': 'stimulus_onset',
    'key_resp.rt': 'reaction_time',
    'block_num': 'task'
}, inplace=True)

# Convert categorical variables to the appropriate type
df['condition_type'] = df['condition_type'].astype('category')
#df['task'] = df['task'].astype('category')
df['task'] = pd.Categorical(df['task'], categories=['1s', '3s', '5s', '10s'], ordered=True)  # Set reference condition here


# Define the halfway point (cutoff time) for each task duration
cutoff_times = {
    '1s': 0.6,
    '3s': 1.5,
    '5s': 2.5,
    '10s': 5.0
}

# Filter the DataFrame for stimulus onsets in the last half of the task ***** CHANGE ME **** 
# < less than cut off 
# > greater than cut off
if half == 'second':
    df_filtered = df[df.apply(lambda row: row['stimulus_onset'] > cutoff_times[row['task']], axis=1)]
else:
    df_filtered = df[df.apply(lambda row: row['stimulus_onset'] < cutoff_times[row['task']], axis=1)]

model_filtered = smf.mixedlm('reaction_time ~ C(task) * C(condition_type) * stimulus_onset', 
                    data=df_filtered, 
                    groups=df_filtered['subject_number']).fit()

# Display the model summary for the filtered data
print(model_filtered.summary())

                              Mixed Linear Model Regression Results
Model:                          MixedLM             Dependent Variable:             reaction_time
No. Observations:               4698                Method:                         REML         
No. Groups:                     26                  Scale:                          0.0089       
Min. group size:                130                 Log-Likelihood:                 4315.9352    
Max. group size:                211                 Converged:                      Yes          
Mean group size:                180.7                                                            
-------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------
Intercept                                         

/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/isaacchristian/opt/anaconda3/envs/decay/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
